## Deboost Bubbles

##### All paths below should be modified

In [ ]:
cd '/home/dpirvu/project/paper_prefactor/'

In [2]:
import os
import sys

sys.path.append('/home/dpirvu/python_stuff/')
sys.path.append('/home/dpirvu/project/paper_prefactor/bubble_codes/')
#sys.path.append('/home/dpirvu/project/paper_prefactor/bubble_codes_older/')
sys.path.remove('/home/dpirvu/DarkPhotonxunWISE/hmvec-master')

print(sys.path)

from plotting import *
from bubble_tools import *
from experiment import *

['/home/dpirvu/project/paper_prefactor', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/python_stuff/', '/home/dpirvu/project/paper_prefactor/bubble_codes/']
['/home/dpirvu/project/paper_prefactor', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/python_stuff/', '/home/dpirvu/project/paper_pr

In [ ]:
%matplotlib inline

In [ ]:
def plot_zoomin(bubble, threshold=2., winsize=100, title=None):
    real = np.copy(bubble)
    real = real[0]
    nT, nN = np.shape(real)
    t_centre, x_centre = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
    tl_stop, tr_stop = int(max(0, t_centre - winsize)), int(min(nT, t_centre + winsize//2))
    xl_stop, xr_stop = int(max(0, x_centre - winsize)), int(min(nN, x_centre + winsize))
    real = real[tl_stop:tr_stop, xl_stop:xr_stop]
    nT, nN = np.shape(real)
    tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
    t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)
    real[np.abs(real) > threshold] = threshold
    simple_imshow([real], x, t, title=title, contour=False, ret=False)
    return

def simple_imshow_continue(ax, bubble, xList, tList, vmin, vmax, contour=False, title=None, aspect='auto', cmap='tab20c'):
    ext = [xList[0],xList[-1],tList[0],tList[-1]]
    im = ax.imshow(bubble, interpolation='none', extent=ext, vmin=vmin, vmax=vmax, aspect=aspect, origin='lower', cmap=cmap)
    if contour:
        ax.contour(xList, tList, bubble, levels=5, linewidths=0.5, colors='k')
    clb = plt.colorbar(im, ax = ax, shrink=0.5)
    ax.plot(0, 0, 'bo', ms=3)
    ax.set_title(title)
    return ax


### Parameters defined in 'experiment' custom module

In [3]:
case = 'minus'
general = get_general_model(case)
tempList, massq, right_Vmax, V, dV, Vinv, nTimeMAX, minSim, maxSim = general

tmp = 2

maxSim = (1000 if tmp == 0 else 2000)
temp, m2, sigmafld = get_model(*general, tmp)
exp_params = [nLat, m2, temp]
print('Experiment', exp_params)

Experiment [2048, 0.7, 0.2]


### Values below depend on shape of potential

In [5]:
# what is a field amplitude that would indicate that a bubble has likely formed
crit_thresh = right_Vmax + 5.*sigmafld; print(crit_thresh)

# should be around crit_thresh, although their precise value is inconsequential
peak_thresh = 2.8
tv_thresh   = 2.5

# exected bubble size at nucleation, in # of lattice sites
crit_rad    = 20

# range of field amplitude for vCOM extraction; see bubble_tools.py module
threshm, threshM = right_Vmax + 0.*sigmafld, right_Vmax + 4.*sigmafld
ampList = np.linspace(threshm, threshM, 12); print([round(ii, 2) for ii in ampList])

# size of window, in lattice sites
winsize = 200

# whether you want to visualise the process
plots = False

2.70793613122138
[1.0, 1.12, 1.25, 1.37, 1.5, 1.62, 1.75, 1.87, 1.99, 2.12, 2.24, 2.37]


In [ ]:
# Set the simulation identifier
sim = 10

try:
    path2sim = bubble_sim_location(*exp_params, sim)
    # Load the full simulation data from the specified path
    real, _ = get_bubble_realisation(nLat, path2sim)

    tdecay = get_decay_time(real)
    real = centre_bubble(real, tdecay)
    
    # Warning: if the velocity is large, you should multiply the bubble (see bubble_tools.py)
    # multiply_bubble(real, phieq, normal, nLat)
    # real = centre_bubble(real, tdecay)

    # Get the dimensions of the full simulation data
    # (nCfull: data columns, nTfull: time steps, nNfull: lattice points)
    fullreal = np.copy(real)
    nCfull, nTfull, nNfull = np.shape(fullreal)
except:
    print('Does not exist')

In [7]:
# Start processing the data
bubble = np.copy(fullreal[0]) # deboosting needs the field component only
bubble = np.abs(bubble)

# It is useful to apply a Gaussian filter to smooth the bubble data
bubble = np.array([gaussian_filter(bubble, 1., mode='nearest')])

nC, nT, nN = np.shape(bubble)
tcen, xcen = find_nucleation_center(bubble[0], phieq, crit_thresh, crit_rad)

# Generate time and spatial coordinate lists centered around the nucleation center
tList = np.linspace(-tcen, nT-1-tcen, nT)
xList = np.linspace(-xcen, nN-1-xcen, nN)


# Initialize a boolean flag as a way to know if an error has appeared; if no error, then bool stays True
bool = True
# Initialize an empty list to store velocities
vellist = []

# Calculate the center-of-mass (COM) velocity and its variance for the bubble
vCOMmeas, varmeas = find_COM_vel(bubble, ampList, winsize, nLat, lightc, phieq, tv_thresh, crit_thresh, crit_rad, plots)

# Check if the calculated COM velocity is NaN, indicating an error
if np.isnan(vCOMmeas):
    print('Dead end at step 0.')
    bool = False


## Note: for "doloop" recursion below:
## It is not always necessary to find the velocity recursively. It is extremely time-consuming.
## It is possible that the first vCOM measurement above was correct.
## Only do this is fluctuations are large,
## or if you want precision (i.e. that the residual velocity be < 0.03c)

# Initialize a step counter
st = 0
# Set a flag to control whether the loop for adjusting the velocity should be executed
doloop = False

# If the loop flag is set to True, enter the loop
if doloop:

    # Continue looping until the absolute value of the COM velocity is less than 0.03 or an error occurs
    while np.abs(vCOMmeas) >= 0.03 and bool:
        # If there are velocities in the list, adjust the simulation based on the collected velocities
        if len(vellist) > 0:
            # Make a copy of the original simulation data
            copy = np.copy(fullreal)[:1]

            # Calculate the total velocity by combining the velocities in the list
            wcop = get_totvel_from_list(vellist)
            # Boost the bubble data using the calculated velocity
            x, t, copy = boost_bubble(copy, tcen, xcen, wcop)
            # Recalculate the COM velocity for the boosted bubble
            vTest, _ = find_COM_vel(copy, ampList, winsize, nLat, lightc, phieq, tv_thresh, crit_thresh, crit_rad, False)
            print('Upon testing this value, we re-measured vTest compared to vCOMmeas', vTest, vCOMmeas)
            # If the new COM velocity is within the acceptable range, update the measured COM velocity
            if np.abs(vTest) < 0.03:
                vCOMmeas = vTest
                break
            # If the new velocity is greater than the last recorded velocity, adjust the measured COM velocity
            if np.abs(vTest) > np.abs(vellist[-1]):
                vCOMmeas = np.sign(vCOMmeas) * random.randint(5, 10) / 10.

        # Increment the step counter
        st += 1
        # Add the current COM velocity to the velocity list and print the step number and velocities
        vellist.append(vCOMmeas)
        print('Step', st, 'vels collected:', vellist)

        # Boost the bubble data using the current COM velocity
        t, x, bubble = boost_bubble(bubble, tcen, xcen, vCOMmeas)

        # Recalculate the COM velocity and variance for the boosted bubble
        vCOMmeas, varmeas = find_COM_vel(bubble, ampList, winsize, nLat, lightc, phieq, tv_thresh, crit_thresh, crit_rad, plots)
        # Check if the recalculated COM velocity is NaN, indicating an error
        if np.isnan(vCOMmeas):
            print('Dead end.')
            bool = False

# If doloop=False but the first detection was successful OR
# if the doloop was exited successfully, proceed to the final steps
if bool:
    print('Final step.')
    # Add the final COM velocity to the velocity list
    vellist.append(vCOMmeas)

    # Calculate the total velocity by relativistic addition
    totvel = get_totvel_from_list(vellist)

    # Plot a zoomed-in view of the original simulation data
    plot_zoomin(np.copy(fullreal), threshold=peak_thresh, winsize=winsize, title='Original zoom')

    # Boost the full simulation data using the calculated total velocity
    t, x, fullreal2save = boost_bubble(np.copy(fullreal), tcen, xcen, totvel)

    # Plot a zoomed-in view of the boosted simulation data
    plot_zoomin(fullreal2save, threshold=peak_thresh, winsize=winsize, title='Final zoom')

    # Generate the path to save the boosted simulation data
    path2RESTsim = rest_sim_location(*exp_params, sim)

    # Note! Sometimes a reflection must be applied, so that all bubbles decay on the same side of the potential

    # Save the simulation ID, boosted simulation data, and total velocity as a numpy array
    np.save(path2RESTsim, np.array([sim, fullreal2save, totvel]))
    print('Saved sim', sim, 'total final velocity, vellist:', totvel, vellist)


[   0    2    6 ... 1994 1996 1997]


### SHOULD PERFORM VISUAL INSPECTION OF EVERY BUBBLE

In [8]:
path2RESTsim  = rest_sim_location(*exp_params, sim)
path2cleansim = clean_sim_location(*exp_params, sim)

win = 200

if os.path.exists(path2cleansim) and os.path.exists(path2RESTsim):
    original = np.load(path2cleansim)
    sim, final, totvel = np.load(path2RESTsim, allow_pickle=True)
    print('Sim ', sim, 'total boost', totvel)

    fig, ax = plt.subplots(1, 2, figsize = (8,3))
    for ii, bubble in enumerate([original, final]):
        real = np.copy(bubble[0])
        nT, nN = np.shape(real)
        
        real = gaussian_filter(real, 1, mode='nearest')
        real[np.abs(real) > crit_thresh] = crit_thresh

        tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
        tl_stop, tr_stop = int(max(0, tcen - win)), int(min(nT, tcen + win//2))
        xl_stop, xr_stop = int(max(0, xcen - win)), int(min(nN, xcen + win))

        real = real[tl_stop:tr_stop, xl_stop:xr_stop]
        nT, nN = np.shape(real)

        tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
        t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)

        if ii == 0: vmin, vmax = np.amin(real), np.amax(real)
        simple_imshow_continue(ax[ii], real, x, t, vmin, vmax, contour=False)

    beautify(ax, times=-70)
    ax[1].grid(which='major', color='black', alpha=1)
    ax[0].grid(which='major', color='black', alpha=1)
    plt.tight_layout()
    plt.show()


[   0    2    6 ... 1994 1996 1997]
